## Introduction to Data Science

### Data Science Tasks: Geographical and Spatial Analysis

In [1]:
#!sudo npm cache clean -f
#!sudo npm install -g n
#!sudo n latest
#!sudo apt-get install --reinstall nodejs-legacy
# ! pip install plotly-geo geopandas shapely pyshp html5lib
#! jupyter labextension install jupyterlab-plotly@4.7.1
#! jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.7.1

In [9]:
# !pip install python-twitter

import pip
pip.main(['install', '-U', 'requests'])
#pip.main(['install','nominatim'])
#pip.main(['install','geopy'])
#pip.main(['install','pydotplus'])
#pip.main(['install','python-twitter'])
#pip.main(['install','python-levenshtein'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)


0

In [13]:
# ! pip install plotly-geo geopandas shapely pyshp html5lib

from __future__ import division

import numpy as np
import pandas as pd
import twitter
import nltk
import re
import networkx as nx
import sys
import os
import json
import time
import urllib
from urllib.request import urlopen
import webbrowser
import codecs
import Levenshtein

from datetime import datetime
from datetime import timedelta
from itertools import chain
from itertools import cycle
from bs4 import BeautifulSoup

from nominatim import Nominatim
import matplotlib.pyplot as plt
from matplotlib import cm
import geopy.geocoders as gg

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display
import ipywidgets as widgets

import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

%matplotlib inline
# Set default figure size for this notebook
plt.rcParams['figure.figsize'] = (16.0, 12.8)
#plt.switch_backend('Agg')

py.offline.init_notebook_mode()

TypeError: '<' not supported between instances of 'module' and 'str'

#### Specifying the path to the files

In [3]:
templates = "../templates/"
outputs = "../outputs/"

dotfile = "graph_retweet.dot"
pngfile = "graph_retweet.png"
protofile = "graph_retweet.html"
tweetsfile = "Tweets_dump.txt"
template_proto = 'template_protoviz.html'

pathdotfile = os.path.join(outputs,dotfile)
pathpngfile = os.path.join(outputs,pngfile)
pathprotofile = os.path.join(outputs,protofile)
pathtweetsfile = os.path.join(outputs,tweetsfile)
pathtemplate = os.path.join(templates,template_proto)

stoplist_en = nltk.corpus.stopwords.words('english')
stoplist_pt = nltk.corpus.stopwords.words('portuguese')
ignorewords = stoplist_en + stoplist_pt + ['',' ','-','rt']

### Using geographical resources within Python

In [4]:
#https://pypi.python.org/pypi/geopy  
#https://www.mapsmarker.com/kb/user-guide/how-mapquest-api/
#https://geopy.readthedocs.io/en/stable/#openmapquest

#geolocator = gg.OpenMapQuest(api_key=OMQapikey)
#geolocator = gg.GoogleV3()

geolocator = gg.Nominatim(user_agent="test_application") #https://operations.osmfoundation.org/policies/nominatim/
#location = geolocator.reverse("52.509669, 13.376294")

To geolocate a query to an address and coordinates:

In [5]:
logradouro = "Dona Mariana, Botafogo"
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Rua Dona Mariana, Botafogo, Zona Sul do Rio de Janeiro, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 22280-020, Brasil
(-22.9530169, -43.1885829)
{'place_id': 84902582, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 14519270, 'boundingbox': ['-22.9566891', '-22.9497369', '-43.1896942', '-43.1872597'], 'lat': '-22.9530169', 'lon': '-43.1885829', 'display_name': 'Rua Dona Mariana, Botafogo, Zona Sul do Rio de Janeiro, Rio de Janeiro, Região Geográfica Imediata do Rio de Janeiro, Região Metropolitana do Rio de Janeiro, Região Geográfica Intermediária do Rio de Janeiro, Rio de Janeiro, Região Sudeste, 22280-020, Brasil', 'class': 'highway', 'type': 'residential', 'importance': 0.41000000000000003}


In [6]:
location = geolocator.reverse('52.509669, 13.376294')
print(location.address)
print((location.latitude, location.longitude))
print(location.raw)

Potsdamer Platz, Bellevuestraße, Botschaftsviertel, Tiergarten, Mitte, Berlin, 10785, Deutschland
(52.5098014, 13.375589791291057)
{'place_id': 235599123, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 3200536, 'lat': '52.5098014', 'lon': '13.375589791291057', 'display_name': 'Potsdamer Platz, Bellevuestraße, Botschaftsviertel, Tiergarten, Mitte, Berlin, 10785, Deutschland', 'address': {'tourism': 'Potsdamer Platz', 'road': 'Bellevuestraße', 'quarter': 'Botschaftsviertel', 'suburb': 'Tiergarten', 'borough': 'Mitte', 'city': 'Berlin', 'postcode': '10785', 'country': 'Deutschland', 'country_code': 'de'}, 'boundingbox': ['52.5082999', '52.5100374', '13.3750548', '13.3769528']}


In [7]:
#Measuring editing distances between names:
Levenshtein.ratio('Dona Mariano, Botafoga', 'Rua Dona Mariana - Botafogo')

0.7755102040816326

In [8]:
#https://developers.google.com/maps/documentation/staticmaps/
#http://stackoverflow.com/questions/2660201/what-parameters-should-i-use-in-a-google-maps-url-to-go-to-a-lat-lon/9919251#9919251
#m – normal map k – satellite h – hybrid p – terrain

def gmap(address,lat,lon,zoom=15,tmap='m'):
    # Google Maps URL template for an iframe
    google_maps_url = 'http://maps.google.com/maps?q={0}&loc:{1}+{2}&z={3}&t={4}&output=embed'.format(address,
                                                                                                     lat,
                                                                                                     lon,
                                                                                                     zoom,
                                                                                                     tmap,)
    display(IFrame(google_maps_url, '800px', '600px'))
    
gmap(address, latitude, longitude,17,'k')

To find the address corresponding to a set of coordinates:

In [9]:
addresses = geolocator.reverse('-15.798,-47.865')
for address in addresses:
    print(address)

Anexo 2 do Senado Federal, N2, Setor de Autarquias Sul, Asa Norte, Plano Piloto, Região Geográfica Imediata do Distrito Federal, Região Integrada de Desenvolvimento do Distrito Federal e Entorno, Região Geográfica Intermediária do Distrito Federal, Distrito Federal, Região Centro-Oeste, 70040906, Brasil
(-15.798067249999999, -47.864397090465545)


### Using Plotly
inspired by [this](https://towardsdatascience.com/an-introduction-to-geographical-data-visualization-3486959cd4b8) post

In [10]:
data = dict (
    type = 'choropleth',
    locations = ['China','Canada','Brazil'],
    locationmode='country names',
    #colorscale = ['Viridis'],
    z=[10,20,30])

In [11]:
outmap = go.Figure(data=[data])
py.offline.plot(outmap)

'temp-plot.html'

Using data on [World Hapiness](https://www.kaggle.com/unsdsn/world-happiness#2017.csv)

In [12]:
df = pd.read_csv('../datasets/CSVs/happiness2017.csv')

data = dict (
    type = 'choropleth',
    locations = df['Country'],
    locationmode='country names',
    #colorscale = ['Viridis'],
    z=df['Happiness.Score'])

outmap = go.Figure(data=[data])
py.offline.plot(outmap)

'temp-plot.html'

Using data on [US Employment](https://raw.githubusercontent.com/plotly/datasets/master/laucnty16.csv) following [these](https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html) and [these instructions](https://plotly.com/python/choropleth-maps/)

In [13]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

fig = px.choropleth(df, 
                    geojson=counties, 
                    locations='fips', 
                    color='unemp',
                    color_continuous_scale="Viridis",
                    range_color=(0, 12),
                    scope="usa",
                    labels={'unemp':'unemployment rate'}
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
py.offline.plot(fig)

'temp-plot.html'

### Using Python and QGIS for geospatial visualizations - a Case Study  
https://www.airpair.com/python/posts/using-python-and-qgis-for-geospatial-visualization  

In [14]:
base_url = "http://www.nuforc.org/webreports/"
index_url = "http://www.nuforc.org/webreports/ndxevent.html"

In [15]:
def can_cast_as_dt(dateStr, fmt):
    try:
        datetime.strptime(dateStr, fmt)
        return True
    except ValueError:
        return False

def parse_dt(dateStr):
    # the data in the website comes in two different formats, try both 
    for fmt in ["%m/%d/%y %H:%M", "%m/%d/%y"]:
        try:
            return datetime.strptime(dateStr, fmt)
        except ValueError:
            continue

def get_data_from_url(url):
    print("Processing {}".format(url))
    data = []
    source = BeautifulSoup(urllib.request.urlopen(url), "html5lib")
    for row in source('tr'):
        if not row('td'):
            continue # header row
        row_data = row('td')
        # parse the datetime from the string
        date_time = parse_dt(row_data[0].text)
        city = row_data[1].text
        state = row_data[2].text
        shape = row_data[3].text
        duration = row_data[4].text
        data.append((date_time, city, state, shape, duration))
    return data

In [16]:
# get the index page
raw_page = urllib.request.urlopen(index_url)
source = BeautifulSoup(raw_page, "html5lib")
# get all the links in the index page
func1 = lambda x: (x.text, base_url + x['href'])
monthly_urls = list(map(func1,source('a')))
# get  the last 12 links that have a text like mm/yyyy
func2 = lambda x: can_cast_as_dt(x[0], "%m/%Y")
last_year_urls = filter(func2, monthly_urls[0:13]) 
# extract the data from each monthly page and flatten the lists of tuples
last_year_ufos = list(chain(*map(lambda x: get_data_from_url(x[1]), last_year_urls)))
# initialize a pandas DataFrame with the list of tuples

Processing http://www.nuforc.org/webreports/ndxe202005.html
Processing http://www.nuforc.org/webreports/ndxe202004.html
Processing http://www.nuforc.org/webreports/ndxe202003.html
Processing http://www.nuforc.org/webreports/ndxe202002.html
Processing http://www.nuforc.org/webreports/ndxe202001.html
Processing http://www.nuforc.org/webreports/ndxe201912.html
Processing http://www.nuforc.org/webreports/ndxe201911.html
Processing http://www.nuforc.org/webreports/ndxe201910.html
Processing http://www.nuforc.org/webreports/ndxe201909.html
Processing http://www.nuforc.org/webreports/ndxe201908.html
Processing http://www.nuforc.org/webreports/ndxe201907.html
Processing http://www.nuforc.org/webreports/ndxe201906.html


In [17]:
ufos_df = pd.DataFrame(last_year_ufos, columns=["start","city","state","shape","duration_description"])

In [18]:
ufos_df.head()

,start,city,state,shape,duration_description
0,2020-05-15 19:25:00,Indore City (India),,,60 seconds
1,2020-05-14 21:40:00,Wellborn,FL,Triangle,5 minutes
2,2020-05-14 20:45:00,Black River Falls,WI,Triangle,2 minutes
3,2020-05-14 18:45:00,Sale (Australia),,Light,10min
4,2020-05-14 03:30:00,Nooksack,WA,Light,5 minutes


In [19]:
ufos_df.replace(to_replace='', value=np.nan, inplace=True, limit=None, regex=False, method='pad')
ufos_df = ufos_df.dropna() 

In [20]:
ufos_df.head()

,start,city,state,shape,duration_description
1,2020-05-14 21:40:00,Wellborn,FL,Triangle,5 minutes
2,2020-05-14 20:45:00,Black River Falls,WI,Triangle,2 minutes
4,2020-05-14 03:30:00,Nooksack,WA,Light,5 minutes
5,2020-05-14 01:59:00,Tempe,AZ,Circle,1.5 seconds
6,2020-05-14 00:20:00,Charleston (James Island),SC,Fireball,5s


#### Engineering time

In [21]:
# function that infers the duration from the text 
def infer_duration_in_seconds(text):
    # try different regexps to extract the total seconds
    text = text.replace('<','')
    text = text.replace('>','')
    text = text.replace('?','')
    text = text.replace('+','')
    text = text.replace('~','')
    metric_text = ["second","s","Second","segundo","minute","m","min","Minute","hour","h","Hour",'Currently']
    metric_seconds = [1,1,1,1,60,60,60,3600,3600,3600,10]
    for metric,mult in zip(metric_text, metric_seconds):
        regex = "\s*(\d+)\+?\s*{}s?".format(metric)
        res = re.findall(regex,text)
        if len(res)>0:
            return int(float(res[0]) * mult)
    return 0

In [22]:
# extract the duration in seconds
ufos_df["duration_secs"] = ufos_df["duration_description"].apply(infer_duration_in_seconds)

# now we can infer the end time of the UFO sighting as well
# which will be useful for the animation later
ufos_df["end"] = ufos_df.apply(lambda x:x["start"] + timedelta(seconds=x["duration_secs"]),axis=1)
ufos_df = ufos_df.dropna()

In [23]:
ufos_df.head()

,start,city,state,shape,duration_description,duration_secs,end
1,2020-05-14 21:40:00,Wellborn,FL,Triangle,5 minutes,300,2020-05-14 21:45:00
2,2020-05-14 20:45:00,Black River Falls,WI,Triangle,2 minutes,120,2020-05-14 20:47:00
4,2020-05-14 03:30:00,Nooksack,WA,Light,5 minutes,300,2020-05-14 03:35:00
5,2020-05-14 01:59:00,Tempe,AZ,Circle,1.5 seconds,5,2020-05-14 01:59:05
6,2020-05-14 00:20:00,Charleston (James Island),SC,Fireball,5s,5,2020-05-14 00:20:05


#### Finding Locations

In [24]:
# http://wiki.openstreetmap.org/wiki/Nominatim_usage_policy
# https://github.com/twain47/Nominatim/blob/master/docs/Installation.md
geolocator = Nominatim()

geolocator.query('Newport News')
#geolocator.query("Houston, TX")

[{'place_id': 235467351,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 206655,
  'boundingbox': ['36.930691', '37.2206658', '-76.646093', '-76.36161'],
  'lat': '36.9786449',
  'lon': '-76.4321089',
  'display_name': 'Newport News, Newport News City, Virginia, United States of America',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.686660556787456,
  'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'},
 {'place_id': 78017695,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 7022584310,
  'boundingbox': ['37.0178211', '37.0278211', '-76.4568619', '-76.4468619'],
  'lat': '37.0228211',
  'lon': '-76.4518619',
  'display_name': 'Newport News, Warwick Boulevard, Huntington Heights, Newport News, Newport News City, Virginia, 23601, United States of America',
  'class': 'ra

In [25]:
#Be careful with too many queries made to the server in a short period of time

ufos_df["lat"] = 0
ufos_df["lon"] = 0
for i in range(len(ufos_df[0:10])):
    try:
        resp_json = geolocator.query(ufos_df['city'][i])
        ufos_df["lat"][i] = resp_json[0]['lat']
        ufos_df["lon"][i] = resp_json[0]['lon']
    except:
        ufos_df["lat"][i] = 0
        ufos_df["lat"][i] = 0
    time.sleep(1.5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is tr

In [26]:
# http://stackoverflow.com/questions/17098654/how-to-store-data-frame-using-pandas-python
ufos_df.to_pickle(os.path.join(outputs,'ufos_df.pkl'))
ufos_df = pd.read_pickle(os.path.join(outputs,'ufos_df.pkl'))
ufos_df.head(10)

,start,city,state,shape,duration_description,duration_secs,end,lat,lon
1,2020-05-14 21:40:00,Wellborn,FL,Triangle,5 minutes,300,2020-05-14 21:45:00,0,-85.8810774
2,2020-05-14 20:45:00,Black River Falls,WI,Triangle,2 minutes,120,2020-05-14 20:47:00,0,-90.8484137
4,2020-05-14 03:30:00,Nooksack,WA,Light,5 minutes,300,2020-05-14 03:35:00,0,-122.32366
5,2020-05-14 01:59:00,Tempe,AZ,Circle,1.5 seconds,5,2020-05-14 01:59:05,0,-111.9400125
6,2020-05-14 00:20:00,Charleston (James Island),SC,Fireball,5s,5,2020-05-14 00:20:05,0,-79.9556455
7,2020-05-13 23:00:00,Golden,MS,Sphere,20 minutes,1200,2020-05-13 23:20:00,0,-91.0176357
9,2020-05-13 21:00:00,Shelton,CT,Light,1 hour,3600,2020-05-13 22:00:00,0,-73.0931641
12,2020-05-13 03:37:00,Springfield,NJ,Light,7 seconds,7,2020-05-13 03:37:07,0,0
13,2020-05-13 00:00:00,Gulf Mills,PA,Other,5 mins,300,2020-05-13 00:05:00,0,0
14,2020-05-12 12:05:00,Granby,CO,Triangle,2 minutes,120,2020-05-12 12:07:00,0,0


In [27]:
# Note: dropna will drop any columns with None values, which is desirable
ufos_df[["start","end","lon","lat","shape"]].dropna().to_csv(os.path.join(outputs,'ufo_data.csv'),
                                                             index=False, 
                                                             encoding="utf-8")